In [4]:
from fastcore.xtras import load_pickle
import pandas as pd 
import os 
from glob import glob
from pathlib import Path
from scipy.stats import sem
import matplotlib.pyplot as plt
from datetime import datetime

import sys
sys.path.append("/home/jorenvanherck/Documents/gptchallengeV2/gptchem-gptj/plotutils/")
from plotutils import *

plt.style.use("/home/jorenvanherck/Documents/gptchallengeV2/gptchem-gptj/plotutils/kevin.mplstyle")

FOLDER = 'out_final'

In [5]:
if not os.path.exists(FOLDER):
    raise FileExistsError('Cannot find folder {}'.format(FOLDER))

all_res = glob(f'{FOLDER}/*.pkl')
print('Pickles found in {}: {}'.format(FOLDER, len(all_res)))

Pickles found in out_final: 10


In [6]:

compiled_res_list = []
conf_mats = []

for results in all_res:
    res_full = load_pickle(results)
    res = res_full['results']
    summary = {
        'size': res_full['train_size'],
        #'representation': res_full['data_summary']['representation'],
        'target': res_full['data_summary']['target'],
        'accuracy': res['accuracy'],
        'f1_macro': res['f1_macro'],
        'f1_micro': res['f1_micro'],
        'kappa': res['kappa'],
        'num_epochs': res_full['config']['tune_settings']['num_train_epochs'],
        'lr': res_full['config']['tune_settings']['learning_rate'],
        'bins': len(set(res['all_y_true'])),
        'test_size': len(res['all_y_true'])
       
    }
    confusion_matrix = {
        'all_y_true':res['all_y_true'],
        'all_y_pred':res['all_y_pred']
    }
    conf_mats.append(confusion_matrix)
    compiled_res_list.append(summary)
print(compiled_res_list)

[{'size': 20, 'target': 'completion_raw', 'accuracy': 0.8333333333333334, 'f1_macro': 0.8285714285714285, 'f1_micro': 0.8333333333333334, 'kappa': 0.6666666666666667, 'num_epochs': 50, 'lr': 0.0003, 'bins': 2, 'test_size': 6}, {'size': 20, 'target': 'completion_raw', 'accuracy': 1.0, 'f1_macro': 1.0, 'f1_micro': 1.0, 'kappa': 1.0, 'num_epochs': 50, 'lr': 0.0003, 'bins': 2, 'test_size': 6}, {'size': 20, 'target': 'completion_raw', 'accuracy': 1.0, 'f1_macro': 1.0, 'f1_micro': 1.0, 'kappa': 1.0, 'num_epochs': 50, 'lr': 0.0003, 'bins': 2, 'test_size': 6}, {'size': 20, 'target': 'completion_raw', 'accuracy': 0.8333333333333334, 'f1_macro': 0.8285714285714285, 'f1_micro': 0.8333333333333334, 'kappa': 0.6666666666666667, 'num_epochs': 50, 'lr': 0.0003, 'bins': 2, 'test_size': 6}, {'size': 20, 'target': 'completion_raw', 'accuracy': 0.8333333333333334, 'f1_macro': 0.8285714285714285, 'f1_micro': 0.8333333333333334, 'kappa': 0.6666666666666667, 'num_epochs': 50, 'lr': 0.0003, 'bins': 2, 'test_

In [7]:
compiled_res = pd.DataFrame(compiled_res_list)
grouped_res =compiled_res.groupby(['bins', 'num_epochs', 'target', 'size']).agg(['mean', 'sem'])


In [8]:
grouped_res


accuracy            f1_macro            \
                                        mean       sem      mean       sem   
bins num_epochs target         size                                          
2    50         completion_raw 20        0.9  0.027217  0.897143  0.027994   

                                    f1_micro           kappa            \
                                        mean       sem  mean       sem   
bins num_epochs target         size                                      
2    50         completion_raw 20        0.9  0.027217   0.8  0.054433   

                                         lr      test_size       
                                       mean  sem      mean  sem  
bins num_epochs target         size                              
2    50         completion_raw 20    0.0003  0.0       6.0  0.0

In [9]:
def get_valid(trues, preds, valid_predictions = [0,1]):
    correct_indices = [i for i, value in enumerate(preds) if value in valid_predictions]

    trues_new = [trues[i] for i in correct_indices]
    preds_new = [preds[i] for i in correct_indices]
    if len(preds) != len(preds_new):
        print(f'{len(preds) - len(preds_new)} removed')
    return trues_new, preds_new

In [12]:
from sklearn.metrics import classification_report
from sklearn import metrics
import os

for csv in os.listdir('predictions_final'):
    try:
        file = os.path.join('predictions_final', csv)
        df = pd.read_csv(file)
        predicted = df.loc[df['partition'] == 'test_taddei']['prediction'].tolist()
        actual = [0,0,1,0,0]
        actual, predicted = get_valid(actual, predicted, [0,1])

        confusion_matrix = metrics.confusion_matrix(actual, predicted)
        #print(classification_report(actual, predicted))

        cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['1','0'])
        cm_display.plot(cmap=plt.cm.Blues)


        plt.savefig('Unseen_test_data.pdf', bbox_inches = 'tight')
    except Exception as e:
        print(e)
    plt.clf()

<Figure size 330x250 with 0 Axes>

<Figure size 330x250 with 0 Axes>

<Figure size 330x250 with 0 Axes>

<Figure size 330x250 with 0 Axes>

<Figure size 330x250 with 0 Axes>

<Figure size 330x250 with 0 Axes>

<Figure size 330x250 with 0 Axes>

<Figure size 330x250 with 0 Axes>

<Figure size 330x250 with 0 Axes>

<Figure size 330x250 with 0 Axes>